In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import glob
import os.path as osp
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import time

In [ ]:
cifar_train = torchvision.datasets.CIFAR10('./',
                                           train=True,
                                           transform=transforms.ToTensor())


# Define CIFAR model architecture

In [ ]:
# source:
# https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        out = F.log_softmax(out, dim=1)
        return out

In [ ]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

# =============================================

In [ ]:
original_model = torchvision.models.resnet18(pretrained=True)
# original_model = torchvision.models.squeezenet1_1(pretrained=True)

class cifar_net(nn.Module):
    def __init__(self, pretrained_resnet):
        super(cifar_net, self).__init__()
        
        self.conv = nn.Conv2d(3, 64, kernel_size=3, padding=1)
        # first layer of pretrained expects input with 64 channels
        self.pretrained = nn.Sequential(*list(pretrained_resnet.children())[4:-2])
        self.fc = nn.Linear(256*8*8, 10)

    def forward(self, x):
        x = F.relu(self.conv(x))
        x = self.pretrained(x)
        x = x.view(x.size(0), -1) # flatten x
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x
    
test_model = cifar_net(original_model)

In [ ]:
cifar_train = torchvision.datasets.CIFAR10('./',
                                           train=True,
                                           transform=transforms.ToTensor())


In [ ]:
s = test_model(torch.rand(1,3,64,64))
s

In [ ]:
s.size()


In [ ]:
test_model.fc.parameters

In [ ]:
test_model.parameters


In [ ]:
256*64*10

In [ ]:
65536./(16*16)

In [ ]:
i=5
cifar_train[i][0].size()
t = cifar_train[i][0].unsqueeze(0)
t.size()




In [ ]:
original_model = torchvision.models.resnet18(pretrained=True)

# original_model = torchvision.models.squeezenet1_1(pretrained=True)


In [ ]:
original_model.parameters()


In [ ]:
a = list(original_model.children())
# a[4:-3]
a[4:]

# =============================================

In [ ]:
resnet_model = ResNet18()
test = resnet_model(cifar_data[5][0].unsqueeze(0))

In [ ]:
torch.exp(test)


# =============================================

In [ ]:
class cifar(nn.Module):
    def __init__(self):
        super(cifar, self).__init__()
        self.dropout_prob = 0.1

        self.conv1 = nn.Conv2d(3, 30, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(30, 30, kernel_size=3, padding=1)

        self.conv3 = nn.Conv2d(30, 40, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(40, 40, kernel_size=3, padding=1)

        self.conv5 = nn.Conv2d(40, 50, kernel_size=3, padding=1)
        self.conv6 = nn.Conv2d(50, 50, kernel_size=3, padding=1)

        self.conv7 = nn.Conv2d(50, 20, kernel_size=3, padding=1)
        self.conv8 = nn.Conv2d(20, 20, kernel_size=3, padding=1)
        
        self.fc1 = nn.Linear(20*2*2, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(self.conv2(self.conv1(x)))
        x = F.max_pool2d( F.dropout(x, p=self.dropout_prob), 2)

        x = F.relu(self.conv4(self.conv3(x)))
        x = F.max_pool2d( F.dropout(x, p=self.dropout_prob), 2)
        
        x = F.relu(self.conv6(self.conv5(x)))
        x = F.max_pool2d( F.dropout(x, p=self.dropout_prob), 2)

        x = F.relu(self.conv8(self.conv7(x)))
        x = F.max_pool2d( F.dropout(x, p=self.dropout_prob), 2)
        
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        return x



# Train model

In [ ]:
# Functionality to save model parameters
def save_checkpoint(checkpoint_path, model, optimizer):
    state = {'state_dict': model.state_dict(),
             'optimizer' : optimizer.state_dict()}
    torch.save(state, checkpoint_path)
    print('model saved to %s' % checkpoint_path)
    
def load_checkpoint(checkpoint_path, model, optimizer):
    state = torch.load(checkpoint_path)
    model.load_state_dict(state['state_dict'])
    optimizer.load_state_dict(state['optimizer'])
    print('model loaded from %s' % checkpoint_path)

In [ ]:
def train(epoch, saved_model_path, log_interval=100):
    start = time.time()
    iteration = 0
    for ep in range(epoch):
        for batch_idx, (data, target) in enumerate(trainset_loader):
            # data, target = data.to(device), target.to(device)
            model.train()  # set training mode
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            if iteration % log_interval == 0:
                print('Total time elapsed (in minutes): {:.2f}'.format( (time.time() - start)/60. ))
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    ep, batch_idx * len(data), len(trainset_loader.dataset),
                    100. * batch_idx / len(trainset_loader), loss.item()))
                test(saved_model_path)
            iteration += 1
    save_checkpoint('{}/final_model'.format(saved_model_path), model, optimizer)
    

In [ ]:
def test(saved_model_path):
    model.eval()  # set evaluation mode
    test_loss = 0
    correct = 0
    best_accuracy = 0.0
    with torch.no_grad():
        for data, target in testset_loader:
            # data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(testset_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(testset_loader.dataset),
        100. * correct / len(testset_loader.dataset)))
    
    if correct/len(testset_loader.dataset) > best_accuracy:
        save_checkpoint('{}/cifar_resnet_best_model'.format(saved_model_path), model, optimizer)
        

# Get data

In [ ]:
cifar_train = torchvision.datasets.CIFAR10('./',
                                           train=True,
                                           transform=transforms.ToTensor())

cifar_test = torchvision.datasets.CIFAR10('./',
                                          train=False,
                                          transform=transforms.ToTensor())

trainset_loader = DataLoader(cifar_train, batch_size=50, shuffle=True, num_workers=1)
testset_loader = DataLoader(cifar_test, batch_size=50, shuffle=True, num_workers=1)

In [ ]:
model = cifar()
# optimizer = optim.Adam(model.parameters())  # original
# optimizer = optim.Adam(model.parameters(), lr=5e-3) # second try
optimizer = optim.Adam(model.parameters())

In [ ]:
# original model accuracies: 34, 38, 40, 42

In [ ]:
# Already trained model for ~ 3 epochs
# After the first 3, test set accuracy was ~65%
# training for another 3

In [ ]:
# Train model and save trained parameters
train(3, 'cifar_saved_models')


# =============================================

In [ ]:
# source:
# https://github.com/kuangliu/pytorch-cifar/blob/master/models/resnet.py

class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, in_planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += x
        out = F.relu(out)
        return out

In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0])
        self.layer2 = self._make_layer(block, 128, num_blocks[1])
        self.layer3 = self._make_layer(block, 256, num_blocks[2])
        self.layer4 = self._make_layer(block, 512, num_blocks[3])
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks):
        layers = list()
        for i in range(num_blocks):
            layers.append(block(self.in_planes, planes))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out

In [ ]:
def ResNet18():
    return ResNet(BasicBlock, [2,2,2,2])

In [ ]:
resnet_model = ResNet18()
# test = resnet_model(cifar_data[5][0].unsqueeze(0))


In [ ]:
cifar_data[5][0].unsqueeze(0).size()

In [ ]:
cifar_data = torchvision.datasets.CIFAR10('./',
                                          train=True,
                                          transform=transforms.ToTensor())


In [ ]:
cifar_data[0][0].size()

i=10
plt.imshow(cifar_data[i][0].numpy().T)
plt.show()
print(cifar_data[i][0].size())


In [ ]:
t = torchvision.models.resnet18(pretrained=True)

In [ ]:
224 - 7 + 2*3

In [ ]:
223//2 + 1

In [ ]:
112/2

In [ ]:
t.parameters